In [7]:
import plotly.express as px
import pandas as pd
import json
import os
os.environ['USER'] = "rmoine"
from pathlib import Path
import numpy as np
from typing import *
from llama.main import compute_metrics_from_files, DatasetName


In [25]:

l = []
for dataset_name in get_args(DatasetName):
    path_data = Path(f"../../data/finetuning/qlora_finetune_{dataset_name}/")
    for dataset_type in ["train","val"]:
        for f in path_data.rglob(f"metrics_epoch_*-0_{dataset_type}*.json"):
            with open(f) as fp:
                d = json.load(fp)
            confusion_matrix = np.array(d['confusion_matrix'])
            accuracy = np.sum(np.diag(confusion_matrix)) / np.sum(confusion_matrix)
            epoch = int(f.stem.split("_")[-2].split("-")[0])
            l.append({"epoch":epoch,"file":dataset_name,"dataset_type":dataset_type,"confusion_matrix":confusion_matrix,"accuracy":accuracy})
df = pd.DataFrame(l)
df["id"] = df["file"] + ", " + df["dataset_type"]
df["accuracy"] = df["accuracy"]*100

In [26]:
px.line(df,x="epoch",y="accuracy",color="id").show()